This tutorial is based on PyTorch's tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html and contains the code snippets from it:  
- device test
- transform, trainset, trainloader, testset, testloader, classes

The license of the original tutorial is the 3-Clause BSD License.  
See LICENSE for detail.


In [ ]:
!pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/b3_proj_2022/MyModules')
import util


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# To monitor the server's GPU installation and usage: log in the server and run `nvidia-smi`.
# It shows the list of GPUs online and their utilization.

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)




In [ ]:
epochs = 6
batch_size_train = 128
batch_size_test = 128
num_shown_images = 8
input_size = 32
# input_size = 64

study_name = "exercise03_02_st01"

In [ ]:
transform_train = transforms.Compose([
    torchvision.transforms.Resize(input_size),
    transforms.RandomCrop(input_size, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.424, 0.415, 0.384), (0.283, 0.278, 0.284))
])
transform_test = transforms.Compose([
    torchvision.transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize((0.424, 0.415, 0.384), (0.283, 0.278, 0.284))
])

trainset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/Colab Notebooks/b3_proj_2022/data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/Colab Notebooks/b3_proj_2022/data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = len(classes)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

# Conv層定義用ユーティリティ関数
def conv_block(ich, och, ksize, num_layers, *, bn=True, pool=False, act=None, **kwargs):
    assert num_layers >= 1
    r = OrderedDict()
    for i in range(num_layers):
        r[str(i)] = nn.Conv2d(ich, och, ksize, **kwargs)
        if bn:
            r["%d-bn" % i] = nn.BatchNorm2d(och)
        ich = och  # set #input_channels to current #output_channels for next loop
    if pool:
        r["pool"] = nn.MaxPool2d(2, 2)
    if act is not None:
        r["act"] = act
    return nn.Sequential(r)


# FC層定義用ユーティリティ関数
def fc_block(ich, och, *, bn=True, pool=False, act=None, **kwargs):
    r = OrderedDict()
    r["1"] = nn.Linear(ich, och, **kwargs)
    if bn:
        r["1-bn"] = nn.BatchNorm1d(och)
    if act is not None:
        r["act"] = act
    return nn.Sequential(r)

# ネット定義
class Net(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(Net, self).__init__()
        
        # Trialからハイパーパラメータをsuggestしてもらう
        num_conv_blocks = 4
        num_conv_layers_per_block = 2
        num_fc_layers = 2
        
        # suggest関数群:
        # suggest_uniform("name", min, max)
        # suggest_loguniform("name", min, max)
        # suggest_categorical("name", [item1, item2, ...])
        # suggest_int("name", min, max)

        if num_fc_layers > 1:
            fc_hidden_size = 1024
        act = nn.ReLU()
        
        # Conv層を実体化
        conv_blocks = []
        conv_blocks.append(
            ("conv1", conv_block(3, 64, 3, 1, bn=True, pool=True, act=act, padding=1, bias=False))
        )
        conv_blocks.append(
            ("conv2", conv_block(64, 128, 3, 1, bn=True, pool=True, act=act, padding=1, bias=False))
        )
        
        # Conv層の実体化 … 上でもらったパラメータを使う
        ich = 128
        och_max = 512
        for i in range(3, num_conv_blocks + 1):
            och = min(ich * 2, och_max)
            conv_blocks.append(
                ("conv%d" % i, conv_block(ich, och, 3, num_conv_layers_per_block, bn=True, pool=True, act=act, padding=1, bias=False))
            )
            ich = och
        
        # FC層の実体化
        fc_blocks = []
        ich = och * (input_size >> num_conv_blocks) ** 2  # och is still in the scope after the previous FOR statement! 気持ち悪い!
        self.fc_input_size = ich
        i = 1
        for _ in range(1, num_fc_layers):
            fc_blocks.append(
                ("fc%d" % i, fc_block(ich, fc_hidden_size, bn=True, act=act, bias=False))
            )
            i += 1
            ich = fc_hidden_size
        fc_blocks.append(
            ("fc%d" % i, fc_block(ich, num_classes, bn=False, act=None, bias=False))  # 最終層だけactとochの扱いが違う
        )
        
        # モデル定義 … Sequentialの利用
        self.conv_blocks = nn.Sequential(OrderedDict(conv_blocks))
        self.fc_blocks = nn.Sequential(OrderedDict(fc_blocks))


    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.view(-1, self.fc_input_size)
        x = self.fc_blocks(x)
        return x



In [ ]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import datetime

# 学習ルーチン
# これを後でループする
def train(net, trial):
    date_str = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
    basename = "%s-%s" % (study_name, date_str)
    
    print("Starting training for '%s'" % basename)
    
    # Learning rate and momentum are also hyper-parameters!
    optim_type = 'adam'  #trial.suggest_....?
    if optim_type == 'adam':
        lr = 1e-3  # trial.suggest_........("lr_adam")
        # suggest結果での条件分岐もOK
        optimizer = optim.Adam(net.parameters(), lr=lr)
    else:
        lr = 1e-3  # trial.suggest_........("lr_sgd")
        momentum=0.9
        optimizer = optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.1)

    dataiter = iter(trainloader)
    images, _ = dataiter.next()
    writer = SummaryWriter("/content/drive/My Drive/Colab Notebooks/b3_proj_2022/runs/%s" % basename)
    writer.add_graph(net, images.to(device))

    criterion = nn.CrossEntropyLoss()

    # Save initial state
    util.add_param(writer, net, 0)

    try:
        for epoch in range(epochs):  # loop over the dataset multiple times
            running_loss = 0.0
            net.train()
            for i, data in enumerate(trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].to(device), data[1].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 100 == 99:
                    train_acc = util.accuracy_batch(outputs, labels)
                    print('[%d, %5d] loss: %.3f, train batch acc: %2d %%' %
                          (epoch + 1, i + 1, running_loss, train_acc))

                    gstep = epoch * len(trainloader) + i
                    writer.add_scalar('Training/Loss', running_loss, gstep)
                    writer.add_scalar('Training/Accuracy', train_acc, gstep)

                    running_loss = 0.0

            # Evaluate intermediate result
            gstep = epoch * len(trainloader) + i
            net.eval()
            with util.IntermediateOutputWriter(writer, net, gstep):
                test_acc = util.accuracy(testloader, net, device=device)
                print('[%d,      ] test acc: %2d %%' %
                      (epoch + 1, test_acc))
            writer.add_scalar('Test/Accuracy', test_acc, gstep)
            util.add_param(writer, net, gstep)

            # 早期打ち止めの判定
            trial.report(test_acc, epoch)
            if trial.should_prune():
                print("Trial is being pruned.")
                raise optuna.exceptions.TrialPruned()
    finally:
        # 打ち止めの場合でも保存はする
        print('Finished Training')

        dirpath = 'saved_models'
        PATH = '%s/%s.pth' % (dirpath, basename)
        try:
            os.mkdir(dirpath)
        except FileExistsError:
            pass
        torch.save(net.state_dict(), PATH)
        trial.set_user_attr("saved_path", PATH)

In [ ]:
# Trialを引数にとる目的関数
# Studyにより１施行当たり1回、自動的にループされる
# Trialからパラメータをsuggestしてもらって実行、結果をreturnする
def objective(trial):
    net = Net(trial, num_classes)
    net.to(device)
    
    try:
        train(net, trial)
    finally:
        net.eval()
        acc = util.accuracy(testloader, net, device=device)
        print("Train finished.\n%.1f %% at %s." % (acc, trial.params))
    return acc


In [ ]:
import sqlite3
current_path = '/content/drive/My Drive/Colab Notebooks/b3_proj_2022/'
db_name ='exercise03_01_st01.db'
con = sqlite3.connect(current_path+db_name)

In [ ]:
import optuna

# Prunerの作成
pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=3, interval_steps=1)
# 途中精度が中央値以下だったら捨てる
# 3 trials揃うまでは判断しない。また各trialで3エポック数えるまで判断しない

# Studyの作成
study = optuna.create_study(
    study_name=study_name,
        # Studyを区別する一意の名前。同じ名前のStudyは同じパラメータ空間に入る
    storage='sqlite:///'+current_path+db_name,
    load_if_exists=True,
        # 同じ名前のStudyがDB上に存在する場合それを読み込み、ない場合新規作成する
        # Falseで同じ名前が存在する時はエラーで止まる
        #
        # | load_if_exists ->  | True               | False              |
        # | ------------------ | ------------------ | ------------------ |
        # | Study is in DB     | Reuse it           | Error              |
        # | Study is not in DB | Create a new study | Create a new study |
    direction='maximize',
        # 最適化は最大化する方向
    pruner=pruner
)

In [ ]:
# 最適化を回す
study.optimize(objective, n_trials=8)

In [ ]:
# 最適な成果を取得
best_trial = study.best_trial

# その時のパラメータと学習済み係数でモデルを復元
net = Net(optuna.trial.FixedTrial(best_trial.params))
net.load_state_dict(torch.load(best_trial.user_attrs['saved_path']))
net.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images[:num_shown_images]
labels = labels[:num_shown_images]

# print images
util.imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(num_shown_images)))

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(num_shown_images)))
accuracy_per_class, accuracy = util.accuracy_of_classes(num_classes, testloader, net)
print('Accuracy of the network on the 10000 test images: %d %%' % accuracy)

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], accuracy_per_class[i]))


In [ ]:
# 結果の出力
# CSVにするのも簡単なので試してみよう

keys = ["##", "#start_date", "#state", "#acc", "saved_path"]
header = ["#", "Start Date", "State", "Acc.", "Saved in"]
keys_set = set(keys)

l = []
for trial in study.trials:
    row = {"##":trial.number, "#start_date":trial.datetime_start, "#state":trial.state, "#acc":trial.value}
    row.update(trial.params)
    row.update(trial.user_attrs)
    l.append(row)
    keys_set.update(row.keys())

keys_rem = list(keys_set - set(keys))
keys.extend(keys_rem)
header.extend(keys_rem)
res = [[record.get(k, None) for k in keys] for record in l]
res.insert(0, header)
util.show_table(res)